In [ ]:
import cv2
import numpy as np
import mysql.connector
import RPi.GPIO as GPIO
import torch
import torch.nn as nn
from torchvision.transforms import transforms

# Подключение к базе данных
db = mysql.connector.connect(
    host="localhost",
    user="username",
    password="password",
    database="database_name"
)

# Установка режима нумерации GPIO
GPIO.setmode(GPIO.BCM)

# Установка пина GPIO для управления сервоприводом
servo_pin = 17  # Измените на соответствующий пин GPIO

# Настройка пина GPIO для управления сервоприводом
GPIO.setup(servo_pin, GPIO.OUT)
servo = GPIO.PWM(servo_pin, 50)  # Задайте частоту ШИМ (в герцах)

# Загрузка обученной модели
model = NumberPlateClassifier()
model.load_state_dict(torch.load('path_to_save_model'))
model.eval()

# Захват видеопотока с камеры (или из файла)
cap = cv2.VideoCapture(0)  # 0 для использования встроенной камеры, либо путь к видеофайлу

while True:
    # Захват кадра из видеопотока
    ret, frame = cap.read()

    # Предобработка кадра, если необходимо
    # ...

    # Применение обработки изображения для распознавания номера автомобиля
    # ...
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((3, 3), np.uint8)
    morphology = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    edged = cv2.Canny(binary, 50, 150)
    gray1 = cv2.GaussianBlur(gray, (5, 5), 0)

    # Применение методов обнаружения краев (например, Canny)
    edged = cv2.Canny(gray1, 50, 150)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    possible_plates = []

    for contour in contours:
        # Оцениваем размер и форму контура
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if 3 < aspect_ratio < 7 and w > 200:  # Подбираем соотношение сторон для номерного знака
            possible_plates.append(contour)

    # Рисуем найденные контуры (потенциальные номерные знаки) на исходном изображении
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.drawContours(frame, possible_plates, -1, (0, 255, 0), 3)
    cv2.imshow('Plate Detection', frame)

    # Получение номеров соответствующих контуров
    found_plates = []
    for i, plate in enumerate(possible_plates):
        x, y, w, h = cv2.boundingRect(plate)
        plate_img = frame[y:y + h, x:x + w]

        # Обработка изображения номера (например, распознавание символов)
        transformed_img = data_transforms(plate_img)
        inputs = torch.unsqueeze(transformed_img, 0).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        plate_number = predicted.item()

        # Поиск номера в базе данных
        cursor = db.cursor()
        cursor.execute("SELECT * FROM license_plate_mapping WHERE id = %s", (plate_number,))
        result = cursor.fetchone()
        if result:
            found_plates.append(result[1])

    # Отправка сигнала на сервопривод для работы
    if found_plates:
        # Запуск сервопривода
        servo.start(0)

        # Изменение положения сервопривода
        duty_cycle = 7.5  # Задайте желаемый цикл работы (в процентах)
        servo.ChangeDutyCycle(duty_cycle)

        # Ждем некоторое время
        # Для удержания сервопривода в данном положении перед выключением
        # Можете изменить и временной интервал, если необходимо
        cv2.waitKey(2000)

        # Остановка сервопривода
        servo.ChangeDutyCycle(0)
        servo.stop()

    # Отображение обработанного кадра
    cv2.imshow('Preprocessed Image', morphology)
    cv2.imshow(' Edged-Display', edged)
    # Выход из цикла при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
db.close()
GPIO.cleanup()